# 서로소 집합 (Disjoint Set)

## 집합의 표현

- 문제 출처: [백준 1717번](https://www.acmicpc.net/problem/1717)

`-` `disjoint-set`을 구현하여 해결할 수 있다

`-` 합집합은 두 집합을 합하면 그만이고 두 원소의 동일 집합 여부는 두 원소의 부모 노드가 동일한지로 판단할 수 있다

In [31]:
UNION = 0
CHECK = 1


def make_set(u):
    p[u] = u  # 각 노드가 자기자신을 가리키게 한다 (u -> u)
    rank[u] = 0


def find_set(u):  # u가 포함된 tree의 부모 노드를 찾아준다
    if p[u] != u:  # u가 자기자신을 가리키지 않으면 (=자식 노드)
        p[u] = find_set(p[u])  # flatten tree, original: (1 -> 3, 3 -> 5, 5 -> 7, 7 -> 7), new: (1 -> 7, 3-> 7, 5 -> 7, 7 -> 7)
    return p[u]


def union_set(u, v):
    uu = find_set(u)
    vv = find_set(v)
    if uu == vv:  # uu와 vv가 같다면 이미 같은 tree에 속하므로 union할 이유가 없다
        return
    rank_u = rank[uu]
    rank_v = rank[vv]
    if rank_u > rank_v:  # v -> u
        p[vv] = uu
    elif rank_u == rank_v:  # v -> u(u -> v도 가능) and rank에 +1
        p[vv] = uu
        rank[vv] += 1
    else:  # u -> v
        p[uu] = vv


def solution():
    global p, rank
    n, m = map(int, input().split())
    p = [i for i in range(n + 1)]
    rank = [0 for _ in range(n + 1)]
    for i in range(n + 1):  # 이미 p를 make_set 적용한 상태로 만들어서 안해도 상관없지만 의미를 분명하게 하려고 추가함
        make_set(i)
    for _ in range(m):
        operator, a, b = map(int, input().split())
        if operator == UNION:
            union_set(a, b)
        else:
            if find_set(a) == find_set(b):
                print("YES")
            else:
                print("NO")


solution()

# input
# 7 8
# 0 1 3
# 1 1 7
# 0 7 6
# 1 7 1
# 0 3 7
# 0 4 2
# 0 1 1
# 1 1 1

 7 8
 0 1 3
 1 1 7


NO


 0 7 6
 1 7 1


NO


 0 3 7
 0 4 2
 0 1 1
 1 1 1


YES


## 거짓말

- 문제 출처: [백준 1043번](https://www.acmicpc.net/problem/1043)

`-` 이 문제는 `union-find` 알고리즘을 통해 해결할 수 있다

`-` 같은 파티에 있는 사람들은 한 배를 탄 것이다

`-` 그 사람들 전체가 진실을 몰라야 해당 파티에서 거짓말을 할 수 있다

`-` 누구 하나라도 진실을 알면 해당 파티에 있는 모든 사람들에게 더 이상 거짓말을 할 수 없다

`-` 그 사람들은 진실을 아는 사람이 되었기 때문에 또 다른 파티에 원래 진실을 아는 사람이 없어도 거짓말을 할 수 없다

`-` 처음에 사람들을 개별 집합으로 초기화한다

`-` 그리고 같은 파티에 있는 사람들을 합친다

`-` 두 명씩 합치면 되며 합집합 연산을 할 때마다 같은 그룹 사람이 한 명 늘어나므로 파티에 $N$명이 있다면 합집합 연산을 $N-1$번 하면 된다

`-` 모든 파티에 대해 합집합 연산을 끝낸 후 원래 진실을 아는 사람을 고려하자

`-` 원래 진실을 아는 사람 각각에 대해 그가 속한 트리의 루트 노드를 set에 추가한다

`-` 파티 하나에 대해 각 참여자들이 속한 트리의 루트 노드가 원래 진실을 아는 사람들 집합에 포함되어 있는지 확인한다

`-` 단 한명이라도 포함되어 있다면 그 파티에서 거짓말을 할 수 없다

`-` 이를 모든 파티에 대해 반복하면 거짓말을 할 수 있는 파티 수의 최댓값을 계산할 수 있다

In [13]:
def make_set(u):
    parent[u] = u
    rank[u] = 0


def find(u):
    if parent[u] != u:
        parent[u] = find(parent[u])
    return parent[u]


def union(u, v):
    root_u = find(u)
    root_v = find(v)
    if root_u == root_v:  # 이미 같은 집합에 속해 있다
        return
    rank_u = rank[root_u]
    rank_v = rank[root_v]
    if rank_u > rank_v:
        parent[root_v] = root_u
    elif rank_u < rank_v:
        parent[root_u] = root_k
    else:
        parent[root_v] = root_u
        rank[root_v] += 1


def solution():
    global parent, rank
    N, M = map(int, input().split())
    parent = [0 for _ in range(N + 1)]  # p[u]는 u가 가리키고 있는 부모 노드
    rank = [0 for _ in range(N + 1)]  # rank[u]는 u가 속한 트리 집합 높이의 상한
    for i in range(1, N + 1):
        make_set(i)
    true_people = list(map(int, input().split()))
    true_people.pop(0)  # 진실을 아는 사람의 수 (필요 없음)
    participants_list = []
    for _ in range(M):
        participants = list(map(int, input().split()))
        n = participants.pop(0)  # 파티에 참여한 사람의 수
        participants_list.append(participants)
        for i in range(n - 1):
            union(participants[i], participants[i + 1])
    true_set = set()
    for t in true_people:
        true_set.add(find(t))
    answer = 0
    for participants in participants_list:
        can_lie = True
        for p in participants:
            if find(p) in true_set:
                can_lie = False
        if can_lie:
            answer += 1
    print(answer)


solution()

# input
# 10 9
# 4 1 2 3 4
# 2 1 5
# 2 2 6
# 1 7
# 1 8
# 2 7 8
# 1 9
# 1 10
# 2 3 10
# 1 4

 10 9
 4 1 2 3 4
 2 1 5
 2 2 6
 1 7
 1 8
 2 7 8
 1 9
 1 10
 2 3 10
 1 4


4


## 사이클 게임

- 문제 출처: [백준 20040번](https://www.acmicpc.net/problem/20040)

`-` 두 점을 연결한다는 것은 두 점 각각과 연결된 원소로 이루어진 집합을 결합한다는 의미이다

`-` 이는 `union-find` 알고리즘을 사용해 해결할 수 있다

`-` 계속해서 주어지는 두 점을 `union` 해나간다

`-` 만약 이미 두 점이 하나의 집합에 포함되어 있다면 사이클이 완성된 것이다 (이는 루트 노드의 일치 여부로 판단 가능하다)

In [14]:
def make_set(u):
    p[u] = u
    rank[u] = 0


def find(u):
    if u != p[u]:
        p[u] = find(p[u])
    return p[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return "no"
    if rank[u_root] < rank[v_root]:
        p[u_root] = v_root
    elif rank[v_root] < rank[u_root]:
        p[v_root] = u_root
    else:
        p[u_root] = v_root
        rank[v_root] += 1
    return "yes"


def solution():
    global p, rank
    n, m = map(int, input().split())
    INF = 2e6
    p = [i for i in range(n)]
    rank = [0 for _ in range(n)]
    for i in range(n):
        make_set(i)
    answer = INF
    for i in range(1, m + 1):
        a, b = map(int, input().split())
        success = union(a, b)
        if success == "yes":
            continue
        answer = min(i, answer)
    if answer == INF:
        answer = 0
    print(answer)


solution()

# input
# 6 5
# 0 1
# 1 2
# 1 3
# 0 3
# 4 5

 6 5
 0 1
 1 2
 1 3
 0 3
 4 5


4


## 여행 가자

- 문제 출처: [백준 1976번](https://www.acmicpc.net/problem/1976)

`-` `disjoint-set`을 사용해 해결할 수 있다

`-` 입력으로 주어진 두 도시를 `union`하고 마지막에 여행을 계획한 도시가 같은 집합에 속하면 여행 가능하다

In [9]:
def make_set(u):
    p[u] = u
    rank[u] = 0


def find(u):
    if p[u] != u:
        p[u] = find(p[u])
    return p[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if rank[v_root] < rank[u_root]:
        p[v_root] = u_root
    elif rank[u_root] < rank[v_root]:
        p[u_root] = v_root
    else:
        p[u_root] = v_root
        rank[u_root] += 1


def solution():
    global p, rank
    N = int(input())
    M = int(input())
    p = [i for i in range(N + 1)]
    rank = [0 for _ in range(N + 1)]
    for u in range(1, N + 1):
        make_set(u)
    for i in range(1, N + 1):
        connection_info = map(int, input().split())
        for j, is_connect in enumerate(connection_info, start=1):
            if is_connect:
                union(i, j)
    cities = list(map(int, input().split()))  # 여행 계획 도시
    answer = "YES"
    root = find(cities[0])
    for city in cities:
        if find(city) != root:
            answer = "NO"
            break
    print(answer)


solution()

# input
# 3
# 3
# 0 1 0
# 1 0 1
# 0 1 0
# 1 2 3

 3
 3
 0 1 0
 1 0 1
 0 1 0
 1 2 3


YES


## 친구 네트워크

- 문제 출처: [백준 4195번](https://www.acmicpc.net/problem/4195)

`-` 서로소 집합을 사용하되 두 집합을 합칠 때 집합의 크기도 합치면 된다

`-` 두 집합을 합칠 때 랭크에 따라 한 집합의 루트 노드를 다른 집합의 루트 노드에 연결한다

`-` 랭크뿐만 아니라 집합의 크기도 기록하는 배열을 만들고 `union`할 때 처리하자

`-` 숫자가 아니라 문자열이 노드를 나타내므로 배열 대신 딕셔너리를 사용하자

In [3]:
def make_set(u):
    p[u] = u
    rank[u] = 0
    size[u] = 1


def find(u):
    if p[u] != u:
        p[u] = find(p[u])
    return p[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if rank[v_root] < rank[u_root]:
        p[v_root] = u_root
        size[u_root] += size[v_root]
    elif rank[u_root] < rank[v_root]:
        p[u_root] = v_root
        size[v_root] += size[u_root]
    else:
        p[u_root] = v_root
        rank[u_root] += 1
        size[v_root] += size[u_root]
    

def solve_testcase():
    global p, rank, size
    F = int(input())
    p = {}
    rank = {}
    size = {}
    for _ in range(F):
        a, b = input().split()
        if a not in p:
            make_set(a)
        if b not in p:
            make_set(b)
        union(a, b)
        print(size[find(a)])  # a와 b는 같은 집합이므로 find(a)와 find(b)는 동일함


def solution():
    T = int(input())
    for _ in range(T):
        solve_testcase()


solution()

# input
# 2
# 3
# Fred Barney
# Barney Betty
# Betty Wilma
# 3
# Fred Barney
# Betty Wilma
# Barney Betty

 2
 3
 Fred Barney


2


 Barney Betty


3


 Betty Wilma


...

2


 Betty Wilma


2


 Barney Betty


4


## 벽 부수고 이동하기 4

- 문제 출처: [백준 16946번](https://www.acmicpc.net/problem/16946)

`-` 간단한 방법은 각 벽에 대해 dfs를 수행하여 이동할 수 있는 칸의 개수를 세는 것이다

`-` $N\times M$의 행렬로 표현되는 맵의 노드는 $N+M$개이고 간선은 $2NM-N-M$개이다

`-` 맵에 벽이 많으면 dfs를 많이 수행하는 대신 이동 반경이 적어진다

`-` 맵의 테두리에 벽이 있다고 하면 벽은 $2N+2M-4$개 존재한다

`-` 각 벽에 대해 dfs를 수행하는 것은 $O\left(N^2M+NM^2\right)$의 시간 복잡도를 가지고 $N,M\le 1000$이므로 시간 초과이다

`-` 맵에 벽이 $2$개 존재한다고 해보자

`-` 하나의 벽에 대해 dfs를 수행하고 나머지 벽에 대해 dfs를 수행한다고 해보자

`-` 이전의 dfs 결과를 바탕으로 어떤 좌표끼리 연결되어 있는지 안다

`-` 나머지 벽을 제외한 모든 공간이 연결되어 있으므로 두 번째 dfs를 수행할 때 모든 공간을 탐색할 필요가 없다

`-` 즉, 공간이 연결되었다는 것은 해당 공간에 속한 임의의 좌표를 방문할 수 있으면 나머지 공간을 모두 방문할 수 있다는 뜻이다

`-` 맵의 각 빈칸에 대해 dfs를 수행하며 방문하는 곳을 하나의 집합으로 관리하자

`-` 이를 union-find 알고리즘을 통해 수행할 것이다

`-` 일단 $2$차원 좌표를 $1$차원 번호로 변환하자

`-` 좌표가 $(x,y)$라면 새로운 번호는 $My + x$이다 ($x,y$는 $0$부터 시작)

`-` 그럼 번호는 $0$부터 $NM-1$까지이다

`-` 벽이 아닌 각 번호에 대해 방문하지 않았다면 dfs를 수행하자

`-` 임의의 공간에 속한 벽이 아닌 좌표 개수를 알기 위해 size 배열을 사용하자

`-` dfs를 수행하면서 만나는 번호에 대해 union을 수행하고 방문 체크를 하자

`-` 만난 번호가 다른 집합이라면 union 과정에서 size도 병합해줘야 한다

`-` 이제 각 벽에 대해 해당 벽을 부수고 이동할 수 있는 곳으로 바꾸고 그 위치에서 이동할 수 있는 칸의 개수를 세어보자

`-` 이는 벽을 기준으로 상하좌우에 위치한 벽이 아닌 공간의 집합의 size 개수 합에 $1$을 더한 것이다 (중복 집합은 제외)

`-` 위와 같이 하면 탐색한 공간은 다시 탐색하지 않으므로 시간 복잡도는 $O(NM)$이 된다

`-` 계속 틀려서 질문 게시판의 반례를 찾아봤다

`-` 메모리 아낄려고 벽 부순 정보의 그래프를 원래 그래프에 덮어 씌우는 방식을 사용했다

`-` 대부분의 경우엔 문제가 안되는데 $10$으로 나눈 나머지가 $0$이면 문제가 된다

`-` 원래는 벽이지만 $0$으로 덮어씌워져서 움직일 수 있는 공간이 되고 이는 인접한 벽에 영향을 끼친다

`-` 모든 벽에 대해 움직일 수 있는 개수를 센 뒤 순회를 다시 하면서 $10$으로 나눴다

In [57]:
import sys

sys.setrecursionlimit(10**6 + 2)


def make_set(u):
    p[u] = u
    rank[u] = 0
    size[u] = 1  # 자기 자신


def find(u):
    if u != p[u]:
        p[u] = find(p[u])
    return p[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if rank[u_root] < rank[v_root]:
        p[u_root] = v_root
        size[v_root] += size[u_root]
    elif rank[u_root] > rank[v_root]:
        p[v_root] = u_root
        size[u_root] += size[v_root]
    else:
        p[v_root] = u_root
        size[u_root] += size[v_root]
        rank[u_root] += 1


def to_1d(x, y):
    return M * y + x


def dfs(x, y, graph, visited):
    num = to_1d(x, y)
    visited.add(num)
    for dx, dy in dxy:
        x_next = x + dx
        y_next = y + dy
        num_next = to_1d(x_next, y_next)
        if num_next in visited:
            continue
        is_in_range = 0 <= x_next < M and 0 <= y_next < N
        need_to_move = is_in_range and graph[y_next][x_next] == BLANK
        if not need_to_move:
            continue
        union(num, num_next)
        dfs(x_next, y_next, graph, visited)


def solution():
    global N, M, BLANK, WALL, dxy, p, rank, size
    N, M = map(int, input().split())
    graph = [list(map(int, list(input()))) for _ in range(N)]
    dxy = [(0, -1), (0, 1), (-1, 0), (1, 0)]
    BLANK = 0
    WALL = 1
    p = [u for u in range(N * M)]
    rank = [0 for _ in range(N * M)]
    size = [1 for _ in range(N * M)]
    visited = set()
    for x in range(M):
        for y in range(N):
            if graph[y][x] == WALL:
                continue
            num = to_1d(x, y)
            make_set(num)
    for x in range(M):
        for y in range(N):
            if graph[y][x] == WALL:
                continue
            num = to_1d(x, y)
            if num in visited:
                continue
            dfs(x, y, graph, visited)
    for x in range(M):
        for y in range(N):
            if graph[y][x] == BLANK:
                continue
            move_count = 1
            visited = set()
            for dx, dy in dxy:
                x_next = x + dx
                y_next = y + dy
                is_in_range = 0 <= x_next < M and 0 <= y_next < N
                if not is_in_range:
                    continue
                if graph[y_next][x_next] != BLANK:
                    continue
                num = to_1d(x_next, y_next)
                root = find(num)
                if root in visited:
                    continue
                visited.add(root)
                move_count += size[root]
            graph[y][x] = move_count
    for x in range(M):
        for y in range(N):
            graph[y][x] %= 10
    for row in graph:
        print("".join(map(str, row)))


solution()

# input
# 4 5
# 11001
# 00111
# 01010
# 10101

 4 5
 11001
 00111
 01010
 10101


46003
00732
06040
50403
